In [3]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import scikeras 
import tensorflow as tf  # Importa o TensorFlow para construção de modelos de   aprendizado de máquina

In [4]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k  # Importa o backend Keras para gerenciamento de sessões
from tensorflow.keras.layers import Dropout  # Importa a camada Dropout
from scikeras.wrappers import KerasClassifier  # Importa KerasClassifier para integração com scikit-learn
from sklearn.model_selection import cross_val_score  # Importa cross_val_score para validação cruzada



In [5]:
eixo_x = pd.read_csv('entradas_breast.csv')
eixo_y = pd.read_csv('saidas_breast.csv')

In [6]:
eixo_x


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.12380,0.18660,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.14440,0.42450,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.00000,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.13740,205.00000,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,111.00000,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.45,26.40,166.10,2027.0,141.00000,0.21130,0.4107,0.2216,206.0000,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,144.00000,0.09791,0.1752,0.05533,...,23.69,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,159.0000,0.05648,...,18.98,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,277.00000,0.35140,152.00000,0.2397,0.07016,...,25.74,39.42,184.60,1821.0,165.00000,0.86810,0.9387,265.0000,0.4087,124.00000


In [7]:
eixo_y

,0
0,0
1,0
2,0
3,0
4,0
...,...
564,0
565,0
566,0
567,0


In [8]:

def criar_rede():
    # Limpa qualquer sessão anterior do Keras para evitar conflitos com modelos anteriores
    k.clear_session()
    
    # Cria uma nova rede neural sequencial
    rede_neural = Sequential([
        # Camada de entrada que recebe dados com 30 características
        tf.keras.layers.InputLayer(shape=(30,)),  # Substitua input_shape por shape
        
        # Primeira camada oculta: camada densa com 16 neurônios e função de ativação ReLU
        tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
        
        # Segunda camada oculta: outra camada densa com 16 neurônios e função de ativação ReLU
        tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
        
        # Camada de saída: camada densa com 1 neurônio para previsão binária
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    # Cria um otimizador Adam com uma taxa de aprendizado e clipping de norma
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=0.5)
    
    # Compila o modelo da rede neural, definindo a função de perda, o otimizador e as métricas
    rede_neural.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])
    
    return rede_neural  # Retorna a rede neural criada


In [9]:
# Cria uma instância de KerasClassifier usando a função criar_rede como estimador
# 'batch_size=10' define o tamanho do lote para o treinamento
# 'epochs=100' especifica o número de épocas para o treinamento
# 'verbose=0' desativa a saída de logs durante o treinamento
rede_neural = KerasClassifier(model=criar_rede, batch_size=10, epochs=100 )

In [10]:
resultados = cross_val_score(estimator=rede_neural, X=eixo_x, y=eixo_y, cv=10, scoring='accuracy')


Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 912us/step - binary_accuracy: 0.6407 - loss: 1.0418
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - binary_accuracy: 0.8077 - loss: 0.4460
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - binary_accuracy: 0.8444 - loss: 0.3792
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - binary_accuracy: 0.8268 - loss: 0.3611
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - binary_accuracy: 0.8795 - loss: 0.3397
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - binary_accuracy: 0.8899 - loss: 0.2913
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - binary_accuracy: 0.8903 - loss: 0.2608
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - binary_accuracy: 0.8865 - loss: 0.3040
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - binary_accuracy: 0.9017 - loss: 0.2753
Epoch 10/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - binary_accuracy: 0.9115 - loss: 0.2496
Epoch 11/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - bina

In [11]:
resultados

array([0.84210526, 0.85964912, 0.87719298, 0.94736842, 0.87719298,
       0.85964912, 0.89473684, 0.85964912, 0.87719298, 0.92857143])

In [12]:
resultados.mean()

0.8823308270676693

In [13]:
resultados.std()

0.031245253281121415

In [17]:
def criar_rede(units=16, learning_rate=0.001, drop_rate=0.2, input_shape=(30,)):
    """
    Cria uma rede neural sequencial para classificação binária.

    Args:
        units (int): Número de unidades nas camadas ocultas.
        learning_rate (float): Taxa de aprendizado do otimizador Adam.
        drop_rate (float): Taxa de dropout para regularização.
        input_shape (tuple): Forma da entrada da rede.

    Returns:
        Sequential: A rede neural configurada.
    """
    k.clear_session()  # Limpa qualquer sessão anterior do Keras para evitar conflitos
    
    # Cria uma nova rede neural sequencial
    rede_neural = Sequential([
        tf.keras.layers.Input(shape=input_shape),  # Camada de entrada com argumento atualizado
        
        # Primeira camada oculta com Dropout
        tf.keras.layers.Dense(units=units, activation='relu', kernel_initializer='he_normal'),
        Dropout(rate=drop_rate),  # Adiciona uma camada Dropout para regularização
        
        # Segunda camada oculta com Dropout
        tf.keras.layers.Dense(units=units, activation='relu', kernel_initializer='he_normal'),
        Dropout(rate=drop_rate),  # Adiciona uma camada Dropout para regularização
        
        # Camada de saída
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    # Cria um otimizador Adam com uma taxa de aprendizado
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=5.0)
    
    # Compila o modelo da rede neural
    rede_neural.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])
    
    return rede_neural  # Retorna a rede neural criada






Epoch 1/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 892us/step - binary_accuracy: 0.6195 - loss: 196.8135 
Epoch 2/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - binary_accuracy: 0.5937 - loss: 120.1092
Epoch 3/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - binary_accuracy: 0.5955 - loss: 89.3236
Epoch 4/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - binary_accuracy: 0.5885 - loss: 60.9120
Epoch 5/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - binary_accuracy: 0.6712 - loss: 39.3509
Epoch 6/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - binary_accuracy: 0.6296 - loss: 25.0654
Epoch 7/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - binary_accuracy: 0.6383 - loss: 19.1859
Epoch 8/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - binary_accuracy: 0.6067 - loss: 13.8052
Epoch 9/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - binary_accuracy: 0.6767 - loss: 10.5273
Epoch 10/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - binary_accuracy: 0.6555 - loss: 8.6537
Epoch 11/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 707u

KeyboardInterrupt: 

In [15]:
# Cria uma instância de KerasClassifier usando a função criar_rede como estimador
# 'batch_size=10' define o tamanho do lote para o treinamento
# 'epochs=100' especifica o número de épocas para o treinamento
# 'verbose=0' desativa a saída de logs durante o treinamento
rede_neural = KerasClassifier(model=criar_rede, batch_size=10, epochs=200)

In [19]:
# Realiza a validação cruzada
resultados = cross_val_score(estimator=rede_neural, X=eixo_x, y=eixo_y, cv=10, scoring='accuracy')  # Avalia a acurácia do modelo

Epoch 1/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 771us/step - binary_accuracy: 0.5075 - loss: 174.3543
Epoch 2/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - binary_accuracy: 0.5538 - loss: 107.5426
Epoch 3/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - binary_accuracy: 0.5162 - loss: 86.0476
Epoch 4/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - binary_accuracy: 0.5620 - loss: 53.0133
Epoch 5/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - binary_accuracy: 0.6405 - loss: 34.6043
Epoch 6/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - binary_accuracy: 0.5897 - loss: 25.1245
Epoch 7/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - binary_accuracy: 0.6432 - loss: 15.6118
Epoch 8/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - binary_accuracy: 0.6282 - loss: 15.8876
Epoch 9/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - binary_accuracy: 0.6560 - loss: 7.9740
Epoch 10/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - binary_accuracy: 0.6320 - loss: 8.5714
Epoch 11/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/

In [20]:
resultados

array([0.89473684, 0.85964912, 0.9122807 , 0.92982456, 0.89473684,
       0.9122807 , 0.89473684, 0.9122807 , 0.89473684, 0.89285714])

In [21]:
media_acuracia = np.mean(resultados)
desvio_padrao_acuracia = np.std(resultados)

print(f"Média da Acurácia: {media_acuracia:.4f}")
print(f"Desvio Padrão da Acurácia: {desvio_padrao_acuracia:.4f}")

Média da Acurácia: 0.8998
Desvio Padrão da Acurácia: 0.0177
